In [1]:
from nnsight import LanguageModel

model = LanguageModel("gpt2", dispatch=True)

/share/u/caden/.conda/envs/work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/share/u/caden/.conda/envs/work/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
with model.trace("Hello, my name is caden", early=True) as tracer:
    model.transformer.h[0].output[0][:] = 0

    model.transformer.h[4].output[0][:] = model.transformer.h[5].output[0]

    # before = model.output.logits.save()

InterventionProtocol_0:[args:('.transformer.h.0.output',1,0,0) l:1 d:0]
getitem_0:[args:(InterventionProtocol_0,0) l:1 d:1]
setitem_0:[args:(getitem_0,slice(None, None, None),0) l:0 d:1]
InterventionProtocol_1:[args:('.transformer.h.5.output',1,0,0) l:1 d:0]
getitem_1:[args:(InterventionProtocol_1,0) l:1 d:1]
InterventionProtocol_2:[args:('.transformer.h.4.output',1,0,0) l:1 d:0]
getitem_2:[args:(InterventionProtocol_2,0) l:1 d:1]
setitem_1:[args:(getitem_2,slice(None, None, None),getitem_1) l:0 d:2]


In [13]:
from typing import List, Set
from nnsight.tracing.Node import Node

def find_root_nodes(nodes: List[Node]) -> List[Node]:
    """Find all root nodes (nodes with no dependencies) in the graph."""
    root_nodes = [node for node in nodes if not node.dependencies]
    return root_nodes

def traverse_node(node: Node, visited: Set[Node] = None) -> None:
    """Traverse through nodes in a graph, processing each node's dependencies and listeners."""
    if visited is None:
        visited = set()

    # Avoid processing a node more than once
    if node.name in visited:
        return node.name

    visited.add(node.name)   

    # Process the current node (this is where you can add any specific logic you need)
    print(f"Processing node: {node.name}")

    # Recursively traverse dependencies
    for dependency in node.dependencies:
        traverse_node(dependency, visited)

    # Recursively traverse listeners
    for listener in node.listeners:
        traverse_node(listener, visited)

def traverse_all_graphs(nodes: List[Node]) -> None:
    """Traverse all nodes in the list, ensuring all subgraphs are covered."""
    visited = set()
    root_nodes = find_root_nodes(nodes)

    for root in root_nodes:
        traverse_node(root, visited)

    # In case there are nodes that are not reachable from any root node
    for node in nodes:
        if node.name not in visited:
            traverse_node(node, visited)

    return visited

graph = tracer._graph
nodes = list(graph.nodes.values())
# visited = traverse_all_graphs(nodes)

In [14]:
visited = traverse_all_graphs(nodes)

Processing node: InterventionProtocol_0
Processing node: getitem_0
Processing node: setitem_0
Processing node: InterventionProtocol_1
Processing node: getitem_1
Processing node: setitem_1
Processing node: getitem_2
Processing node: InterventionProtocol_2


In [4]:
with model.trace("Hello, my name is caden", graph=tracer._clone) as tracer:
    # pass
    after = model.output.logits.save()

ValueError: Accessing value before it's been set.

In [ ]:
after

ValueError: Accessing value before it's been set.